<a href="https://colab.research.google.com/github/nkikn/Facial-Expression-Recognition/blob/main/facial_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [39]:
!pip install kaggle
!pip install wandb torch torchvision pandas numpy matplotlib seaborn

In [40]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [41]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [42]:
# Download the dataset
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

challenges-in-representation-learning-facial-expression-recognition-challenge.zip: Skipping, found more recently modified local copy (use --force to force download)
replace example_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [43]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# import torch
# import torch.nn as nn
# import torch.optim as optim
# import torch.nn.functional as F
# from torch.utils.data import Dataset, DataLoader
# import torchvision.transforms as transforms
# from sklearn.metrics import classification_report, confusion_matrix
# import wandb
# import time
# from datetime import datetime
# from sklearn.model_selection import train_test_split

In [44]:
# Facial Expression Recognition - Simple CNN Baseline
# Notebook 1: Simple CNN Architecture

# Cell 1: Setup and Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import wandb
import os
from tqdm import tqdm
import time


In [45]:
# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Cell 2: Wandb Setup
# Initialize Wandb
wandb.login()

# Wandb configuration
config = {
    'model_name': 'test_overfit',
    'learning_rate': 0.001,
    'batch_size': 128,
    'epochs': 20,
    'optimizer': 'Adam',
    'loss_function': 'CrossEntropyLoss',
}

# Initialize wandb run
run = wandb.init(
    project="facial-expression-recognition",
    name="test_overfit",
    config=config,
)


Using device: cuda


In [64]:
import pandas as pd
import numpy as np
from torch.utils.data import Dataset
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader


class My_Set(Dataset):
    def __init__(self, dataframe, transform=None):
        self.data = dataframe.reset_index(drop=True)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        pixels = self.data.iloc[idx]['pixels']
        image = np.array([int(x) for x in pixels.split()]).reshape(48, 48)
        image = Image.fromarray(image.astype(np.uint8), mode='L')

        if self.transform:
            image = self.transform(image)

        label = self.data.iloc[idx]['emotion']
        return image, label


# Define transforms
transform = transforms.Compose([
    # transforms.ToPILImage(),
    transforms.Resize((48, 48)),
    transforms.ToTensor(),
])

from sklearn.model_selection import train_test_split


train_df = pd.read_csv('train.csv')
train_indices, val_indices = train_test_split(
    range(len(train_df)),
    test_size=0.2,
    stratify=train_df['emotion'],
    random_state=42
)



train_dataset = My_Set(train_df, transform)
val_dataset = My_Set(train_df, transform)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=config['batch_size'],
    shuffle=True,
    num_workers=2,
    # sampler=sampler
)

val_loader = DataLoader(
    val_dataset,
    batch_size=config['batch_size'],
    shuffle=False,
    num_workers=2
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")


Training samples: 28709
Validation samples: 28709


In [65]:
import torch.nn as nn

class SimpleCNN(nn.Module):
    def __init__(self, num_classes=7):
        super(SimpleCNN, self).__init__()

        # First conv layer: input 1 channel, output 32 channels
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)  # 1x48x48 -> 32x48x48
        self.pool = nn.MaxPool2d(2, 2)  # Halve spatial size

        # Second conv layer: input 32 channels, output 64 channels
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1) # 32x24x24 -> 64x24x24

        # After 2 poolings, spatial size 48 -> 24 -> 12
        # So flatten dimension = 64 * 12 * 12
        self.fc1 = nn.Linear(64 * 12 * 12, 128)
        self.fc2 = nn.Linear(128, num_classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)  # 48 -> 24

        x = self.relu(self.conv2(x))
        x = self.pool(x)  # 24 -> 12

        x = x.view(-1, 64 * 12 * 12)  # Flatten

        x = self.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [66]:

# Initialize model
model = SimpleCNN().to(device)

# Count parameters
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")


Total parameters: 1,199,495
Trainable parameters: 1,199,495


In [67]:

# Log model info to wandb
wandb.log({"total_parameters": total_params, "trainable_parameters": trainable_params})

# Cell 5: Training Setup
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.1)


In [68]:

# Cell 6: Training Functions
def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    pbar = tqdm(train_loader, desc="Training")
    for inputs, labels in pbar:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        pbar.set_postfix({
            'Loss': f'{running_loss/len(train_loader):.4f}',
            'Acc': f'{100*correct/total:.2f}%'
        })

    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100 * correct / total

    return epoch_loss, epoch_acc


In [69]:

# Validation function
def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            running_loss += loss.item()
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

            all_preds.extend(predicted.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    epoch_loss = running_loss / len(val_loader)
    epoch_acc = 100.0 * correct / total

    return epoch_loss, epoch_acc, all_preds, all_targets

In [70]:

# Cell 7: Training Loop
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, num_epochs):
    train_losses = []
    train_accuracies = []
    val_losses = []
    val_accuracies = []

    best_val_acc = 0.0

    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 10)

        start_time = time.time()

        # Training phase
        train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)

        # Validation phase
        val_loss, val_acc, val_predictions, val_labels = validate_epoch(model, val_loader, criterion, device)

        # Update learning rate
        scheduler.step()

        # Calculate epoch time
        epoch_time = time.time() - start_time

        # Store metrics
        train_losses.append(train_loss)
        train_accuracies.append(train_acc)
        val_losses.append(val_loss)
        val_accuracies.append(val_acc)

        # Log to wandb
        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_accuracy": train_acc,
            "val_loss": val_loss,
            "val_accuracy": val_acc,
            "learning_rate": optimizer.param_groups[0]['lr'],
            "epoch_time": epoch_time
        })

        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'best_simple_cnn.pth')
            wandb.log({"best_val_accuracy": best_val_acc})

        print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        print(f"Epoch Time: {epoch_time:.2f}s")

        # Early stopping check (optional)
        if epoch > 20 and val_acc < best_val_acc - 5:  # Stop if val acc drops by 5%
            print("Early stopping triggered")
            break

    return train_losses, train_accuracies, val_losses, val_accuracies, val_predictions, val_labels


In [71]:

# Cell 8: Train Model
print("Starting training...")
train_losses, train_accs, val_losses, val_accs, final_predictions, final_labels = train_model(
    model, train_loader, val_loader, criterion, optimizer, scheduler, config['epochs']
)


Starting training...

Epoch 1/30
----------


Training: 100%|██████████| 449/449 [00:28<00:00, 15.91it/s, Loss=1.6765, Acc=33.25%]


Train Loss: 1.6765, Train Acc: 33.25%
Val Loss: 1.5542, Val Acc: 40.35%
Epoch Time: 55.09s

Epoch 2/30
----------


Training: 100%|██████████| 449/449 [00:28<00:00, 15.74it/s, Loss=1.4788, Acc=43.14%]


Train Loss: 1.4788, Train Acc: 43.14%
Val Loss: 1.4122, Val Acc: 45.45%
Epoch Time: 53.12s

Epoch 3/30
----------


...

Train Loss: 0.1819, Train Acc: 95.69%
Val Loss: 0.1680, Val Acc: 96.20%
Epoch Time: 52.88s

Epoch 29/30
----------


Training: 100%|██████████| 449/449 [00:26<00:00, 17.16it/s, Loss=0.1742, Acc=95.93%]


Train Loss: 0.1742, Train Acc: 95.93%
Val Loss: 0.1600, Val Acc: 96.56%
Epoch Time: 51.71s

Epoch 30/30
----------


Training: 100%|██████████| 449/449 [00:28<00:00, 16.03it/s, Loss=0.1662, Acc=96.15%]


Train Loss: 0.1662, Train Acc: 96.15%
Val Loss: 0.1517, Val Acc: 96.75%
Epoch Time: 51.38s


In [ ]:

# Cell 9: Results Analysis and Visualization
# Plot training curves
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 3, 2)
plt.plot(train_accs, label='Train Accuracy')
plt.plot(val_accs, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()


In [ ]:

# Confusion Matrix
plt.subplot(1, 3, 3)
class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
cm = confusion_matrix(final_labels, final_predictions)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('Actual')

plt.tight_layout()
plt.savefig('simple_cnn_results.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:

# Log plots to wandb
wandb.log({"training_curves": wandb.Image(plt)})

# Cell 10: Detailed Performance Analysis
# Classification Report
print("\nClassification Report:")
print(classification_report(final_labels, final_predictions, target_names=class_names))

# Calculate per-class accuracy
class_accuracies = {}
for i, class_name in enumerate(class_names):
    class_mask = np.array(final_labels) == i
    if np.sum(class_mask) > 0:
        class_acc = np.sum(np.array(final_predictions)[class_mask] == i) / np.sum(class_mask)
        class_accuracies[class_name] = class_acc * 100

print("\nPer-class Accuracies:")
for class_name, acc in class_accuracies.items():
    print(f"{class_name}: {acc:.2f}%")

# Log final metrics to wandb
final_metrics = {
    "final_train_accuracy": train_accs[-1],
    "final_val_accuracy": val_accs[-1],
    "best_val_accuracy": max(val_accs),
    "final_train_loss": train_losses[-1],
    "final_val_loss": val_losses[-1],
}

# Add per-class accuracies
for class_name, acc in class_accuracies.items():
    final_metrics[f"class_accuracy_{class_name.lower()}"] = acc

wandb.log(final_metrics)


In [ ]:

# # Cell 11: Model Summary and Insights
# print("\n" + "="*50)
# print("SIMPLE CNN MODEL SUMMARY")
# print("="*50)
# print(f"Architecture: {config['architecture']}")
# print(f"Total Parameters: {total_params:,}")
# print(f"Best Validation Accuracy: {max(val_accs):.2f}%")
# print(f"Final Validation Accuracy: {val_accs[-1]:.2f}%")
# print(f"Training Time: {len(train_accs)} epochs")

# print("\nKey Observations:")
# print("1. Simple architecture with only 2 conv layers")
# print("2. No regularization techniques applied")
# print("3. Basic architecture serves as baseline")
# print("4. Limited capacity may cause underfitting")
# print("5. Good starting point for progressive development")

# print("\nNext Steps:")
# print("1. Increase model depth (more conv layers)")
# print("2. Add regularization (BatchNorm, Dropout)")
# print("3. Experiment with different architectures")
# print("4. Apply data augmentation")


In [ ]:

# Cell 12: Save Model and Artifacts
# Save final model
torch.save(model.state_dict(), 'test_overfit.pth')

# Save model artifact to wandb
artifact = wandb.Artifact('simple_cnn_model', type='model')
artifact.add_file('test_overfit.pth')
wandb.log_artifact(artifact)

# Save training history
import pickle
history = {
    'train_losses': train_losses,
    'train_accuracies': train_accs,
    'val_losses': val_losses,
    'val_accuracies': val_accs,
    'config': config,
    'final_predictions': final_predictions,
    'final_labels': final_labels
}


with open('test_overfit_history.pkl', 'wb') as f:
    pickle.dump(history, f)

# Finish wandb run
wandb.finish()

print("\nTraining completed and logged to Wandb!")
print("Model saved as 'test_overfit.pth'")
print("Training history saved as 'test_overfit_history.pkl'")

**Overfit_test**

In [72]:
wandb.finish()


best_val_accuracy,▃▅▆▁▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇█████████████
epoch,▁▁▁▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▅▅▅▅▆▆▆▆▇▇▇▇███
epoch_time,▁▁▁█████▇█▇█▇█▇▇█▇█▇▇▇▇█▇█▇▇█▇▇▇▇
learning_rate,█████████████████▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁
total_parameters,▁▁▁
train_accuracy,▄▅▆▁▂▃▃▃▄▄▄▅▅▅▆▆▆▇▇▇█████████████
train_loss,▃▃▃█▇▆▆▆▅▅▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
trainable_parameters,▁▁▁
val_accuracy,▃▅▆▁▂▂▃▃▄▄▄▅▅▆▆▆▇▇▇▇█████████████
val_loss,▄▃▂█▇▇▆▆▅▅▅▄▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁
best_val_accuracy,96.74666
